## This script loads images that were already downloaded from mapillary to a sqlite3 database, formats them for a user to be able to search by criteria

Do not sure for now (9/29/2024)

In [2]:
import requests
from dotenv import load_dotenv
from os import getenv

import pandas as pd
import numpy as np

import sqlite3
import json
import re

import sys
local_modules = os.path.join(os.getcwd(), '..','..','src')
sys.path.append(local_modules)
import importlib
import mapillary_utils as mu
importlib.reload(mu)
import image_processing_utils as ipu
importlib.reload(ipu)
#from image_db_interface import DatabaseManager as idi
import image_db_interface as idi
importlib.reload(idi)

import image_and_detection_selector as ids
importlib.reload(ids)

from datetime import datetime
import uuid

In [3]:
target_database = "360_images_2024-09-08.db"
db_path = f"d://projects_working_directories//202408_pano_images//{target_database}"

In [4]:
db = idi.DatabaseManager(db_path)

# Connect to the database
db.connect()


sql = """select * from sampled_images"""
df_images = db.fetch_data(sql)

sql = """select * from image_detections"""
df_detections = db.fetch_data(sql)



Connected to d://projects_working_directories//202408_pano_images//360_images_2024-09-08.db


In [5]:
df_images.sample()

,image_lat,image_lon,residual,image_id,camera_type,is_pano,camera_focal_len,camera_k1,camera_k2,image_path,...,image_url,altitude,compass_angle,exif_orientation,make,model,height,width,country,city
9395,-6.285296,106.880181,76.612049,725775799401766,spherical,1,None,NaN,NaN,None,...,https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6...,50.636,286.32177340774,1,GoPro,GoPro Max,2880,5760,Indonesia,Jakarta


In [6]:
df_detections.sample()

,image_id,detection_label,feature_count,detection_prct_of_image
46078,336891745764366,void--unlabeled,81,4


In [7]:
## note sure where this function should go...
def zz_select_detections(df_detections, selected_detection):
    """
    Some detections are a little ambiguous as to their meaning. This function
    helps clarify what certain detections means.

    You can pass in a string with as much or little precision as you want.
    If you pass in low precision (eg. "construction") it will match with all
    detections that begin with construction
    """

    def _get_matched_detections(unqiue_detections, selected_detection):
        matched_detections = []
        for detection in unqiue_detections:
            if re.match(selected_detection,detection):
                matched_detections.append(detection)
        return matched_detections
        


    unique_detections = list(set(df_detections.detection_label.tolist()))
    matched_detections = _get_matched_detections(unique_detections, selected_detection)

    return matched_detections


selected_detections = ids.select_detections(df_detections, 'construction--fl')
    

In [8]:
def zz_find_images_matching(df_detections, selected_detections, threshold=5, match_strat='threshold'):
    """
    For each image, if it matches all the passed in detections, return a true

    match_strat is either "all" or "threshold"
    if "all" then only images that match all the detetections will be returned
    if "threshold" then as long as the collective threshold is met, whether it be
    one of 10 detections, the image will be returned

    In ALL cases, images that don't meet the threshold will be ignored

    """

    print(selected_detections)

    #first, drop all detections under the threshold
    print(df_detections.shape)
    #temp_df = df_detections[df_detections.detection_prct_of_image >= threshold]
    #print(temp_df.shape)

    #first, reduce df to only selected detections
    temp_df = df_detections[df_detections['detection_label'].isin(selected_detections)]
    print(temp_df.shape)

    #for each image, get the detection label count matched and the total percent of image taken
    temp_images = temp_df[['image_id','detection_label','detection_prct_of_image']].groupby('image_id').agg(
        count_of_detections=('detection_label', 'size'),
        sum_of_detections=('detection_prct_of_image', 'sum')
    ).reset_index()

    #drop images where the aggregate detection prct doesn't meet threshold
    temp_images = temp_images[temp_images.sum_of_detections >= threshold]
    
    if match_strat=='threshold':
        matched_images = temp_images.image_id.tolist()
    elif match_strat=='all':
        matched_images = temp_images.image_id[temp_images.count_of_detections == len(selected_detections)].tolist()
    else:
        print('unexpected match_strat passed... returning threshold')
        matched_images = temp_images.image_id.tolist()

    return matched_images
            
        
matched_images = ids.find_images_matching(df_detections, selected_detections, threshold=10, match_strat='threshold')
len(matched_images)

['construction--flat--parking', 'construction--flat--curb-cut', 'construction--flat--crosswalk-plain', 'construction--flat--sidewalk', 'construction--flat--road-shoulder', 'construction--flat--driveway', 'construction--flat--pedestrian-area', 'construction--flat--service-lane', 'construction--flat--parking-aisle', 'construction--flat--traffic-island', 'construction--flat--rail-track', 'construction--flat--bike-lane', 'construction--flat--road']
(227443, 4)
(22146, 4)


5346

In [98]:
selected_detections = ['object--banner', 'object--sign--advertisement', 'object--sign--back', 
                       'object--sign--information', 'object--sign--other', 'object--sign--store']

In [102]:
df_images.image_url[df_images.image_id.isin(matched_images)].tolist()


['https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An_D94rwqrfg5iBZAepgu60rDAu5VurROliBKMvmS6szIT_OurJzl6-zhTbOcNL6PytacPrXoOin3OqETDTFaV9pMlh719iQX_HfbkClO9KQTYt_frMUqkOEFS1esWcxyLDHCtOwAKx2smFgfbBZ52g?ccb=10-5&oh=00_AYAC0Yp3RyfSpeDaDRlW1ExMltZUP6AnVsDY_39ReDjUSw&oe=67053CF9&_nc_sid=201bca',
 'https://scontent.ffsd3-1.fna.fbcdn.net/m1/v/t6/An-h9hb4_T1SCHg5ME5zqeG5prdgF5L-I7pG0Q3KHCLJwu0NzyLs_wzAA9Uw-11F0jKoRwT3fiJqy9gsUt_myscqJoiy6srjoOpI8tjDMjgMv10Bp60wNNKg725OKlmiSmzG8vAACBUqy8LHAdk_Cjs?ccb=10-5&oh=00_AYA1JPEfirAFs0DFeCsKAV5sXXtpczp9JrWbpZUem5qfFg&oe=67055D8A&_nc_sid=201bca']

In [65]:
selected_detections

['construction--flat--traffic-island',
 'construction--flat--road',
 'construction--flat--parking',
 'construction--flat--driveway',
 'construction--flat--parking-aisle',
 'construction--flat--sidewalk',
 'construction--flat--curb-cut',
 'construction--flat--service-lane',
 'construction--flat--bike-lane',
 'construction--flat--crosswalk-plain',
 'construction--flat--road-shoulder',
 'construction--flat--pedestrian-area',
 'construction--flat--rail-track']

In [ ]:
busy_market = ['human','object--sign--store','construction']


In [55]:
df_detections[['image_id','detection_label','detection_prct_of_image']].groupby('image_id').count()

,detection_label,detection_prct_of_image
image_id,,
1000138537907833,26,26
1000146154569642,26,26
1000255234893447,24,24
1000320658379856,29,29
1000407131205067,24,24
...,...,...
999617694644292,21,21
999626310736555,24,24
999667991668183,20,20


In [6]:
set(df_detections.detection_label.tolist())



{'animal--bird',
 'animal--ground-animal',
 'complementary--chevron-left--g1',
 'complementary--chevron-right--g1',
 'complementary--obstacle-delineator--g3',
 'construction--barrier--concrete-block',
 'construction--barrier--curb',
 'construction--barrier--fence',
 'construction--barrier--guard-rail',
 'construction--barrier--road-median',
 'construction--barrier--road-side',
 'construction--barrier--separator',
 'construction--barrier--temporary',
 'construction--barrier--wall',
 'construction--flat--bike-lane',
 'construction--flat--crosswalk-plain',
 'construction--flat--curb-cut',
 'construction--flat--driveway',
 'construction--flat--parking',
 'construction--flat--parking-aisle',
 'construction--flat--pedestrian-area',
 'construction--flat--rail-track',
 'construction--flat--road',
 'construction--flat--road-shoulder',
 'construction--flat--service-lane',
 'construction--flat--sidewalk',
 'construction--flat--traffic-island',
 'construction--structure--bridge',
 'construction--s